In [0]:
import cv2
import os
import numpy as np
from matplotlib import pyplot as plt
import gc
import random

In [0]:
import h5py
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten
from keras.optimizers import Adam
from keras.losses import mean_squared_error as MSE
from keras.callbacks import ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split



Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
os.chdir("/content/drive/My Drive/comma_challenge/")
os.chdir("./data")


In [0]:
#helper code to check RAM and GPU resources

#!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
#!pip install gputil
#!pip install psutil
#!pip install humanize
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn’t guaranteed
gpu = GPUs[0]
def printm():
 process = psutil.Process(os.getpid())
 print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " | Proc size: " + humanize.naturalsize( process.memory_info().rss))
 print("GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB".format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
printm()

  Running setup.py bdist_wheel for gputil ... - done
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built gputil
Gen RAM Free: 12.8 GB  | Proc size: 328.2 MB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [0]:
path_to_imgs = "/content/drive/My Drive/comma_challenge/data/flow_imgs_2_hsv2_resized/"
path_to_test_imgs = "/content/drive/My Drive/comma_challenge/data/flow_test_imgs/"
path_to_trainY = "/content/drive/My Drive/comma_challenge/data/train.txt"
file_paths = sorted([os.path.join(path_to_imgs,f) for f in os.listdir(path_to_imgs)])
file_paths_test = sorted([os.path.join(path_to_test_imgs,f) for f in os.listdir(path_to_test_imgs)])

trainYs = []

with open(path_to_trainY, "r") as train_file:
  trainYs = train_file.read().splitlines()

trainYs = [float(line) for line in trainYs]

trainYs = np.asarray(trainYs[1:-1])
(train_files, test_files, trainY, testY) = train_test_split(file_paths,
	trainYs, test_size=0.20, random_state=42)

print(len(train_files))
print(len(test_files))


16318
4080


In [0]:
#adding frames from test images where the car is stopped (540 frames)
train_files.extend(file_paths_test[1098:1638])
trainY = np.concatenate((trainY, np.zeros([540])))
print(len(train_files))
print(len(trainY))


16858
16858


In [0]:
trainX = np.zeros([len(train_files), 74,220, 3])
testX = np.zeros([len(test_files), 74,220,3])


In [0]:
import os
from io import BytesIO
import tarfile
import tempfile
from six.moves import urllib

from matplotlib import gridspec
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image

import tensorflow as tf

(16318, 74, 220, 1)
(4080, 74, 220, 1)


In [0]:
def draw_hsv(flow):
    h, w = flow.shape[:2]
    fx, fy = flow[:,:,0], flow[:,:,1]
    ang = np.arctan2(fy, fx) + np.pi
    v = np.sqrt(fx*fx+fy*fy)
    hsv = np.zeros((h, w, 3), np.uint8)
    hsv[...,0] = ang*(180/np.pi/2)
    hsv[...,1] = 255
    hsv[...,2] = np.minimum(v*4, 255)
    bgr = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    return bgr

In [0]:
import os
import cv2
import numpy as np
from matplotlib import pyplot as plt

#resize parameters
newWidth, newHeight = 220, 74

#cropping parameters
top = 0
bottom = 370
left = 0
right = 640

#path to video
#used for both train and test videos, currently set to train video
path_to_vid = "/content/drive/My Drive/comma_challenge/data/train.mp4"

#path to file to store optical flow images for every instant
#used for both train and test dataset, currently set to train dataset
path_to_flow = "/content/drive/My Drive/comma_challenge/data/flow_images_2_hsv2_resized"

cap = cv2.VideoCapture(path_to_vid)
ret, frame1 = cap.read()
ret, frame2 = cap.read()

prvs = frame1[top:bottom, left:right]
prvs = cv2.resize(prvs, (newWidth, newHeight), interpolation=cv2.INTER_AREA)
prvs = cv2.cvtColor(prvs,cv2.COLOR_BGR2GRAY)

curr = frame2[top:bottom, left:right]
curr = cv2.resize(curr, (newWidth, newHeight), interpolation=cv2.INTER_AREA)
curr = cv2.cvtColor(curr,cv2.COLOR_BGR2GRAY)

i = 2

In [0]:
# getting all the frames from the video and calculating optical flow
while(1):
    ret, frame3 = cap.read()
    if(not ret):
        break
    
    next_ = frame3[top:bottom, left:right]
    next_ = cv2.resize(next_, (newWidth, newHeight), interpolation=cv2.INTER_AREA)
    next_ = cv2.cvtColor(next_,cv2.COLOR_BGR2GRAY)
    flow = cv2.calcOpticalFlowFarneback(prvs,next_, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    hsv2 = draw_hsv(flow)
    
    name = ""
    if i < 10:
        name = "0000"
    elif i < 100:
        name = "000"
    elif i < 1000:
        name = "00"
    elif i < 10000:
        name = "0"
    else:
        name = ""
    
    name += str(i)
    cv2.imwrite(os.path.join(path_to_flow, "hsv_" + name + ".png"),hsv2)
    
    prvs = curr
    curr = next_
    i += 1
cap.release()
cv2.destroyAllWindows()

'\n#plt.imshow(trainX[1])\nprint(file_paths[1])\nimg = cv2.imread(file_paths[1],cv2.IMREAD_COLOR)\n#plt.imshow(img)\n#plt.imshow(trainX[1])\ntrainX[1] = np.asarray(img)\nimg_as = np.asarray(img)\nplt.imshow(trainX[1])\nassert np.array_equal(trainX[1], img_as)\ntrainX[1]\n#img_as\n'

In [0]:
!ls

checkpoints		      model_with_dropout.json
checkpoints_bothflow	      model_with_dropout_vert.json
flow_imgs_2_hsv2_resized      test1.txt
flow_imgs_2_hsv2_resized.tar  test.mp4
flow_test_imgs		      test.txt
flow_test_imgs.tar	      test_vert_best.txt
horz_flow_imgs		      test_vert.txt
horz_flow_imgs.tar	      train.mp4
horz_flow_test_imgs	      train.txt
horz_flow_test_imgs.tar       vert_flow_imgs
logs			      vert_flow_imgs.tar
model_with_dropout_both.h5    vert_flow_test_imgs
model_with_dropout_both.json  vert_flow_test_imgs.tar
model_with_dropout.h5


In [0]:
i = 0
for file in train_files:
  trainX[i] = cv2.imread(file, cv2.IMREAD_COLOR)
  if(i % 100 == 0):
    print(i)
  i += 1
print(i)
i = 0
for file in test_files:
  testX[i] = cv2.imread(file, cv2.IMREAD_COLOR)
  if(i % 100 == 0):
    print(i)
  i += 1
print(i)

In [0]:
os.chdir("/content/drive/My Drive/comma_challenge/")
os.chdir("./data")
!mkdir checkpoints_withtest

In [0]:
# helper code to save large files in Google drive
# Which file to send?
file_name = "LARGE_FILE"

from google.colab import auth
from googleapiclient.http import MediaFileUpload
from googleapiclient.discovery import build

auth.authenticate_user()
drive_service = build('drive', 'v3')

def save_file_to_drive(name, path):
  file_metadata = {'name': name, 'mimeType': 'application/octet-stream'}
  media = MediaFileUpload(path, mimetype='application/octet-stream', resumable=True)
  created = drive_service.files().create(body=file_metadata, media_body=media, fields='id').execute()
  
  return created

save_file_to_drive(file_name, file_name)

{'id': '16L5sB89lIhPeEFGdV4XsDLuFNmWOTbK1'}

In [0]:
trainX /= 255.0
testX /= 255.0


In [0]:
# output was gotten (i.e, this cell was run) after model was trained and checkpoints were created
!ls ./checkpoints_withtest/

weights-improvement-01-8.68.hdf5  weights-improvement-11-1.16.hdf5
weights-improvement-02-5.85.hdf5  weights-improvement-12-0.90.hdf5
weights-improvement-03-4.02.hdf5  weights-improvement-13-0.89.hdf5
weights-improvement-04-3.31.hdf5  weights-improvement-15-0.86.hdf5
weights-improvement-05-3.11.hdf5  weights-improvement-16-0.67.hdf5
weights-improvement-06-1.67.hdf5  weights-improvement-20-0.46.hdf5
weights-improvement-08-1.29.hdf5


In [0]:
#Model hyperparameters

NUM_EPOCHS=25
opt='adam'
los='mse'
minibatch_size=32

In [0]:
#model architecture

model = Sequential()
model.add(Conv2D(24, (5,5), activation='relu', strides=2, data_format='channels_last', input_shape=(74,220,3)))
model.add(Conv2D(36, (5,5), activation='relu', strides=2, data_format='channels_last'))
model.add(Dropout(0.5))
model.add(Conv2D(48, (5,5), activation='relu', strides=2, data_format='channels_last'))
model.add(Dropout(0.5))
model.add(Conv2D(64, (3,3), activation='relu', data_format='channels_last'))
model.add(Conv2D(64, (3,3), activation='relu', data_format='channels_last'))
model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(50, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))


In [0]:
checkpoint_path = "/content/drive/My Drive/comma_challenge/data/checkpoints_withtest/weights-improvement-{epoch:02d}-{val_loss:.2f}.hdf5"
model_path = "/content/drive/My Drive/comma_challenge/data/model_with_dropout_withtest.json"
log_path="/content/drive/My Drive/comma_challenge/logs"
os.chdir("/content/drive/My Drive/comma_challenge/data/")

In [0]:
!ls

checkpoints		      model_with_dropout.json
checkpoints_bothflow	      model_with_dropout_vert.json
checkpoints_withtest	      model_with_dropout_withtest.json
flow_imgs_2_hsv2_resized      test1.txt
flow_imgs_2_hsv2_resized.tar  test_bothflow_best.txt
flow_test_imgs		      test.mp4
flow_test_imgs.tar	      test.txt
horz_flow_imgs		      test_vert_best.txt
horz_flow_imgs.tar	      test_vert.txt
horz_flow_test_imgs	      train.mp4
horz_flow_test_imgs.tar       train.txt
logs			      vert_flow_imgs
model_with_dropout_both.h5    vert_flow_imgs.tar
model_with_dropout_both.json  vert_flow_test_imgs
model_with_dropout.h5	      vert_flow_test_imgs.tar


In [0]:
adam = Adam(lr=1e-4, beta_1=1, beta_2=0.999, epsilon=1e-08, decay=0.0)
model.compile(optimizer="adam", loss=los, metrics=['mse', 'acc'])
checkpt = ModelCheckpoint(checkpoint_path, monitor='val_loss', verbose=1, save_best_only=True, mode='min', period=1)
tboard = TensorBoard(log_dir=log_path, write_graph=False)
callbacks_list = [checkpt]


In [0]:
model.fit(trainX, trainY, validation_data=(testX, testY), epochs=NUM_EPOCHS, batch_size=minibatch_size, callbacks=callbacks_list)

Train on 16858 samples, validate on 4080 samples
Epoch 1/25
16858/16858 [==============================] - 23s 1ms/step - loss: 30.6962 - mean_squared_error: 30.6962 - acc: 0.0054 - val_loss: 8.6842 - val_mean_squared_error: 8.6842 - val_acc: 2.4510e-04

Epoch 00001: val_loss improved from inf to 8.68417, saving model to /content/drive/My Drive/comma_challenge/data/checkpoints_withtest/weights-improvement-01-8.68.hdf5
Epoch 2/25
16858/16858 [==============================] - 18s 1ms/step - loss: 8.4988 - mean_squared_error: 8.4988 - acc: 0.0135 - val_loss: 5.8498 - val_mean_squared_error: 5.8498 - val_acc: 0.0012

Epoch 00002: val_loss improved from 8.68417 to 5.84979, saving model to /content/drive/My Drive/comma_challenge/data/checkpoints_withtest/weights-improvement-02-5.85.hdf5
Epoch 3/25
16858/16858 [==============================] - 18s 1ms/step - loss: 5.7024 - mean_squared_error: 5.7024 - acc: 0.0233 - val_loss: 4.0189 - val_mean_squared_error: 4.0189 - val_acc: 0.0000e+00

Epo

In [0]:
os.chdir("/content/drive/My Drive/comma_challenge/data/")
model_json = model.to_json()
with open(model_path, "w") as file:
  file.write(model_json)

In [0]:
img = testX[10]
img = img.reshape(1,img.shape[0], img.shape[1], img.shape[2])
pred = model.predict(img)

In [0]:
pred

array([[14.604296]], dtype=float32)

In [0]:
testY[10]

13.95459

In [0]:
!ls

checkpoints		      model_with_dropout.json
checkpoints_bothflow	      model_with_dropout_vert.json
flow_imgs_2_hsv2_resized      test1.txt
flow_imgs_2_hsv2_resized.tar  test_bothflow_best.txt
flow_test_imgs		      test.mp4
flow_test_imgs.tar	      test.txt
horz_flow_imgs		      test_vert_best.txt
horz_flow_imgs.tar	      test_vert.txt
horz_flow_test_imgs	      train.mp4
horz_flow_test_imgs.tar       train.txt
logs			      vert_flow_imgs
model_with_dropout_both.h5    vert_flow_imgs.tar
model_with_dropout_both.json  vert_flow_test_imgs
model_with_dropout.h5	      vert_flow_test_imgs.tar


In [0]:
!ls "./flow_test_imgs" | wc -l

10796


In [0]:
file_paths_test = sorted([os.path.join(path_to_test_imgs,f) for f in os.listdir(path_to_test_imgs)])
test_paths = file_paths_test


In [0]:
from keras.models import model_from_json

os.chdir("/content/drive/My Drive/comma_challenge/data")
json_file = open('model_with_dropout_withtest.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("./checkpoints_withtest/weights-improvement-20-0.46.hdf5")
print("Loaded model from disk")

loaded_model.compile(loss='mse', optimizer='adam')

Loaded model from disk


In [0]:
#final test file. Changed the name to test.txt manually later
with open("test_withtest_best.txt", 'w') as f:
  for file in test_paths:
    img = cv2.imread(file, cv2.IMREAD_COLOR) / 255.0
    img = np.reshape(img, [1, img.shape[0], img.shape[1], img.shape[2]])
    pred = max(loaded_model.predict(img)[0,0], 0.0) 
    pred_str = str(pred)
    f.write(pred_str + '\n')